In [ ]:
import pandas as pd
from dmyplant2 import cred, MyPlant, Engine, msgFSM, plot_with_additional_results
from pprint import pprint as pp
    
cred()
mp = MyPlant(0)
Engine._list_cached_validations();

In [ ]:
e=Engine.from_sn(mp,1486174)
modes = ['???','OFF','MANUAL','AUTO']
success = [True,False]
e['IB Site Name'] + ' ' + e['Engine ID'], modes, success

In [ ]:
e=Engine.from_sn(mp,1486174)
fsm = msgFSM(e, p_from=e['Commissioning Date'], p_to='2022-01-15', successtime=300)
fsm.run1(enforce=True) # run Finite State Machine

In [ ]:
rda = fsm.starts
rda['stime'] = rda.apply(lambda x: x['starttime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda['etime'] = rda.apply(lambda x: x['endtime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda[['no','success','mode','stime','etime','startpreparation','starter','speedup','idle','synchronize','loadramp','cumstarttime','targetoperation','rampdown','coolrun','runout']].round(2)

In [ ]:
vset = ['Various_Values_SpeedAct','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif','Hyd_PressOil','Hyd_TempOil']
dset = [{'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue'},
        #{'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange'},
        #{'col':['Hyd_PressOilDif'],'ylim': [0, 2], 'color':'black'},
        {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown'},
        {'col':['Hyd_TempOil'],'ylim': [0, 110], 'color':'#2171b5'}]
startversuch = rda.iloc[6]; 
plot_with_additional_results(fsm,startversuch, vset, dset, dfigsize=(16,10)) 


In [ ]:
startversuch=fsm.starts.iloc[6].to_dict()
pp(startversuch)

In [ ]:
# calc phase durations
# targetoperation, zusammemspielen aller Einträge
tlr = startversuch['timing']['targetoperation']
tlr = [{'start':tlr[0]['start'], 'end':tlr[-1]['end']}]
startversuch['timing']['targetoperation'] = tlr
phases = list(startversuch['timing'].keys())
durations = { ph:pd.Timedelta(startversuch['timing'][ph][-1]['end'] - startversuch['timing'][ph][-1]['start']).total_seconds() for ph in phases}
durations['cumstarttime'] = sum([v for k,v in durations.items() if k in ['startpreparation','starter','speedup','idle','synchronize','loadramp']])
startversuch.update(durations)
pp(startversuch)

#### phases = [x[6:] for x in self.results['starts'][-1]['timing'] if x.startswith('start_')]
# durations = { ph:pd.Timedelta(self.results['starts'][-1]['timing']['end_'+ph] - self.results['starts'][-1]['timing']['start_'+ph]).total_seconds() for ph in phases}
# durations['cumstarttime'] = sum([v for k,v in durations.items() if k in ['startpreparation','starter','speedup','idle','synchronize','loadramp']])
# self.results['starts'][-1].update(durations)
# if 'targetoperation' in self.results['starts'][-1]:
#     #successful if the targetoperation run was longer than specified
#     self.results['starts'][-1]['success'] = (self.results['starts'][-1]['targetoperation'] > self._successtime)
# self.results['starts'][-1]['count_alarms'] = len(self.results['starts'][-1]['alarms'])
# self.results['starts'][-1]['count_warnings'] = len(self.results['starts'][-1]['warnings'])

In [ ]:
fsm._runlogdetail[350:]
[x for x in fsm._runlogdetail if x.startswith('*')][50:]